<a href="https://colab.research.google.com/github/codeseeboy/beverage-bottle-detection-model/blob/main/Beverage_bottle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Mount Google Drive and Install Dependencies
import os
import zipfile
import shutil
from pathlib import Path
import yaml
import torch
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Install required packages
!pip install ultralytics roboflow supervision

# Import required libraries
from ultralytics import YOLO
import cv2
from IPython.display import Image, display

print("✅ Setup completed!")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# Define project paths
PROJECT_ROOT = "/content/drive/MyDrive/BeverageDetection"
DATASET_PATH = f"{PROJECT_ROOT}/dataset"
MODELS_PATH = f"{PROJECT_ROOT}/models"
CHECKPOINTS_PATH = f"{MODELS_PATH}/checkpoints"
BEST_MODELS_PATH = f"{MODELS_PATH}/best_models"
RUNS_PATH = f"{PROJECT_ROOT}/runs"

# Create directories if they don't exist
os.makedirs(PROJECT_ROOT, exist_ok=True)
os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(CHECKPOINTS_PATH, exist_ok=True)
os.makedirs(BEST_MODELS_PATH, exist_ok=True)
os.makedirs(RUNS_PATH, exist_ok=True)

print(f"📁 Project structure created at: {PROJECT_ROOT}")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Step 2: Extract and Prepare Dataset
def extract_dataset(zip_path, extract_to):
    """Extracts the dataset from a zip file to the specified directory."""
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Dataset extracted to: {extract_to}")
        return True
    except Exception as e:
        print(f"Error extracting dataset: {e}")
        return False

def prepare_data_yaml(dataset_path):
    """Modifies data.yaml to include absolute paths for train, val, and test directories."""
    yaml_path = os.path.join(dataset_path, "data.yaml")

    if os.path.exists(yaml_path):
        with open(yaml_path, 'r') as file:
            data = yaml.safe_load(file)

        # Set absolute paths for training, validation, and test data
        data['train'] = os.path.join(dataset_path, 'train/images')
        data['val'] = os.path.join(dataset_path, 'valid/images')
        data['test'] = os.path.join(dataset_path, 'test/images')

        with open(yaml_path, 'w') as file:
            yaml.dump(data, file, default_flow_style=False)

        print("Updated data.yaml with correct paths")
        print("Dataset details:")
        print(f"  - Classes: {data['nc']}")
        print(f"  - Train images: {data['train']}")
        print(f"  - Val images: {data['val']}")
        print(f"  - Test images: {data['test']}")

        return yaml_path, data
    else:
        print(f"data.yaml not found at {yaml_path}")
        return None, None

# Extract the dataset zip file
ZIP_FILE = "/content/drive/MyDrive/BeverageDetection/dataset/data.zip"

if os.path.exists(ZIP_FILE):
    extract_success = extract_dataset(ZIP_FILE, DATASET_PATH)
    if extract_success:
        yaml_path, dataset_info = prepare_data_yaml(DATASET_PATH)
else:
    print(f"Dataset zip file not found at: {ZIP_FILE}")
    print("Please upload the dataset zip file and update the ZIP_FILE variable accordingly.")

# Verify the dataset directory structure
def verify_dataset_structure(dataset_path):
    """Checks for the presence of required folders and counts contained files."""
    required_folders = ['train/images', 'train/labels', 'valid/images', 'valid/labels']

    for folder in required_folders:
        folder_path = os.path.join(dataset_path, folder)
        if os.path.exists(folder_path):
            count = len(os.listdir(folder_path))
            print(f"{folder}: {count} files")
        else:
            print(f"Missing folder: {folder}")

print("\nVerifying dataset structure:")
verify_dataset_structure(DATASET_PATH)


✅ Dataset extracted to: /content/drive/MyDrive/BeverageDetection/dataset
✅ Updated data.yaml with correct paths
📊 Dataset info:
   - Classes: 73
   - Train images: /content/drive/MyDrive/BeverageDetection/dataset/train/images
   - Val images: /content/drive/MyDrive/BeverageDetection/dataset/valid/images
   - Test images: /content/drive/MyDrive/BeverageDetection/dataset/test/images

📋 Dataset Structure Verification:
✅ train/images: 3828 files
✅ train/labels: 3828 files
✅ valid/images: 1094 files
✅ valid/labels: 1094 files


In [ ]:
import os
import time
import json
import shutil
import torch
from datetime import datetime
from ultralytics import YOLO
from IPython.display import display, Image

class BeverageDetectionTrainer:
    def __init__(self, dataset_yaml_path, project_root):
        self.dataset_yaml_path = dataset_yaml_path
        self.project_root = project_root
        self.checkpoints_path = f"{project_root}/models/checkpoints"
        self.best_models_path = f"{project_root}/models/best_models"
        self.training_log_path = f"{project_root}/training_log.json"

        # Create directories if they don't exist
        os.makedirs(self.checkpoints_path, exist_ok=True)
        os.makedirs(self.best_models_path, exist_ok=True)

    def save_training_state(self, epoch, model_path, metrics):
        """Save training state for resuming"""
        state = {
            'epoch': epoch,
            'model_path': model_path,
            'metrics': metrics,
            'timestamp': datetime.now().isoformat(),
            'dataset_path': self.dataset_yaml_path
        }

        try:
            with open(self.training_log_path, 'w') as f:
                json.dump(state, f, indent=2)
            print(f"💾 Training state saved at epoch {epoch}")
        except Exception as e:
            print(f"⚠️ Could not save training state: {e}")

    def load_training_state(self):
        """Load previous training state"""
        if os.path.exists(self.training_log_path):
            try:
                with open(self.training_log_path, 'r') as f:
                    state = json.load(f)
                print(f"📂 Found previous training state from epoch {state['epoch']}")
                return state
            except Exception as e:
                print(f"⚠️ Could not load training state: {e}")
        return None

    def train_model(self, epochs=50, img_size=640, batch_size=16, resume=True):
        """Train YOLO model with checkpointing"""

        # Validate dataset path
        if not os.path.exists(self.dataset_yaml_path):
            raise FileNotFoundError(f"Dataset YAML not found: {self.dataset_yaml_path}")

        # Check for previous training state
        previous_state = None
        if resume:
            previous_state = self.load_training_state()

        # Initialize model
        if previous_state and os.path.exists(previous_state['model_path']):
            print(f"🔄 Resuming training from {previous_state['model_path']}")
            model = YOLO(previous_state['model_path'])
            start_epoch = previous_state['epoch']
        else:
            print("🚀 Starting fresh training")
            model = YOLO('yolov8n.pt')  # Start with pre-trained weights
            start_epoch = 0

        # Detect device
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"🖥️  Using device: {device}")

        # Training parameters
        train_params = {
            'data': self.dataset_yaml_path,
            'epochs': epochs,
            'imgsz': img_size,
            'batch': batch_size,
            'device': device,
            'workers': 2 if device == 'cpu' else 4,
            'project': self.project_root,
            'name': 'beverage_detection_training',
            'save_period': 10,  # Save checkpoint every 10 epochs
            'patience': 50,     # Early stopping patience
            'save': True,
            'val': True,
            'plots': True,
            'verbose': True,
            'cache': True,      # Cache images for faster training
            'optimizer': 'SGD', # SGD or Adam
            'lr0': 0.01,        # Initial learning rate
            'momentum': 0.937,  # SGD momentum
            'weight_decay': 0.0005,
            'warmup_epochs': 3.0,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1
        }

        print(f"🎯 Training Configuration:")
        for key, value in train_params.items():
            print(f"   {key}: {value}")

        try:
            # Start training
            print(f"\n⏰ Training started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            results = model.train(**train_params)

            # Get the best model path
            best_model_source = results.save_dir / 'weights' / 'best.pt'
            best_model_path = f"{self.best_models_path}/best_beverage_model.pt"

            # Copy the best model
            if os.path.exists(best_model_source):
                shutil.copy(str(best_model_source), best_model_path)
                print(f"📁 Best model saved to: {best_model_path}")

            # Extract final metrics safely
            final_metrics = {}
            try:
                if hasattr(results, 'box'):
                    final_metrics = {
                        'map50': float(results.box.map50) if hasattr(results.box, 'map50') else 0.0,
                        'map': float(results.box.map) if hasattr(results.box, 'map') else 0.0,
                        'precision': float(results.box.mp) if hasattr(results.box, 'mp') else 0.0,
                        'recall': float(results.box.mr) if hasattr(results.box, 'mr') else 0.0
                    }
            except Exception as e:
                print(f"⚠️ Could not extract metrics: {e}")
                final_metrics = {'error': str(e)}

            # Save final training state
            self.save_training_state(epochs, best_model_path, final_metrics)

            print(f"\n🎉 Training completed successfully!")
            print(f"⏰ Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

            if final_metrics and 'error' not in final_metrics:
                print(f"📈 Final Metrics:")
                print(f"   mAP@0.5: {final_metrics.get('map50', 'N/A'):.4f}")
                print(f"   mAP@0.5:0.95: {final_metrics.get('map', 'N/A'):.4f}")
                print(f"   Precision: {final_metrics.get('precision', 'N/A'):.4f}")
                print(f"   Recall: {final_metrics.get('recall', 'N/A'):.4f}")

            return model, results

        except KeyboardInterrupt:
            print(f"\n⏸️  Training interrupted by user at {datetime.now().strftime('%H:%M:%S')}")
            self._save_interrupted_state(model, start_epoch)
            raise

        except Exception as e:
            print(f"❌ Training error: {e}")
            self._save_interrupted_state(model, start_epoch)
            raise e

    def _save_interrupted_state(self, model, start_epoch):
        """Save state when training is interrupted"""
        try:
            current_epoch = getattr(model.trainer, 'epoch', start_epoch) if hasattr(model, 'trainer') else start_epoch
            current_model_path = f"{self.checkpoints_path}/interrupted_epoch_{current_epoch}.pt"

            # Try to save the current model state
            if hasattr(model, 'trainer') and hasattr(model.trainer, 'last'):
                if os.path.exists(model.trainer.last):
                    shutil.copy(model.trainer.last, current_model_path)
                    self.save_training_state(current_epoch, current_model_path, {})
                    print(f"💾 Progress saved to: {current_model_path}")
                    print(f"🔄 Resume by running the training again")
        except Exception as e:
            print(f"⚠️ Could not save interrupted state: {e}")

    def validate_model(self, model_path=None):
        """Run validation on trained model"""
        if model_path is None:
            model_path = f"{self.best_models_path}/best_beverage_model.pt"

        if not os.path.exists(model_path):
            print(f"❌ Model not found: {model_path}")
            return None

        print(f"🔍 Running validation on: {model_path}")
        model = YOLO(model_path)
        results = model.val(data=self.dataset_yaml_path)

        print(f"📊 Validation Results:")
        print(f"   mAP@0.5: {results.box.map50:.4f}")
        print(f"   mAP@0.5:0.95: {results.box.map:.4f}")
        print(f"   Precision: {results.box.mp:.4f}")
        print(f"   Recall: {results.box.mr:.4f}")

        return results

# Main training execution
def run_training(dataset_yaml_path, project_root, epochs=50, img_size=640, batch_size=16):
    """Main function to run the complete training pipeline"""

    print("🚀 YOLO Beverage Detection Training Pipeline")
    print("=" * 50)

    # Initialize trainer
    trainer = BeverageDetectionTrainer(dataset_yaml_path, project_root)

    # Display configuration
    print(f"⚙️  Training Configuration:")
    print(f"   Dataset: {dataset_yaml_path}")
    print(f"   Project Root: {project_root}")
    print(f"   Epochs: {epochs}")
    print(f"   Image Size: {img_size}")
    print(f"   Batch Size: {batch_size}")
    print(f"   GPU Available: {torch.cuda.is_available()}")
    print(f"   CUDA Version: {torch.version.cuda if torch.cuda.is_available() else 'N/A'}")

    try:
        # Start training
        model, results = trainer.train_model(
            epochs=epochs,
            img_size=img_size,
            batch_size=batch_size,
            resume=True
        )

        # Display training plots if available
        plots_dir = f"{project_root}/beverage_detection_training"
        if os.path.exists(plots_dir):
            print(f"\n📊 Training plots saved in: {plots_dir}")

            # Try to display results plot
            results_plot = f"{plots_dir}/results.png"
            if os.path.exists(results_plot):
                try:
                    display(Image(results_plot))
                except:
                    print(f"📈 Results plot available at: {results_plot}")

        # Run final validation
        print(f"\n🔍 Running final validation...")
        trainer.validate_model()

        return trainer, model, results

    except KeyboardInterrupt:
        print("\n⏸️  Training interrupted by user")
        print("💾 Progress has been saved. Run again to resume.")
        return trainer, None, None

    except Exception as e:
        print(f"\n❌ Training failed: {e}")
        print("💾 Progress has been saved. Check the error and run again to resume.")
        raise e

# Direct execution section - combines both your original codes
# This replaces both Step 3 and Step 4 from your original code

# Configuration - Updated for your directory structure
PROJECT_ROOT = "/content/drive/MyDrive/BeverageDetection"
yaml_path = "/content/drive/MyDrive/BeverageDetection/dataset/data.yaml"  # This should be created from your unzipped data

# Training parameters
EPOCHS = 50
IMG_SIZE = 640
BATCH_SIZE = 16  # Reduce to 8 or 4 if you get GPU memory errors

# Check if required variables exist (from your original code logic)
if 'yaml_path' in locals() and yaml_path and yaml_path != "path_to_your_dataset.yaml":
    # Initialize trainer (from your Step 3)
    trainer = BeverageDetectionTrainer(yaml_path, PROJECT_ROOT)
    print("✅ Trainer initialized and ready!")

    # Start training (from your Step 4)
    print("\n🚀 Starting YOLO Training...")
    print(f"⚙️  Configuration:")
    print(f"   Epochs: {EPOCHS}")
    print(f"   Image Size: {IMG_SIZE}")
    print(f"   Batch Size: {BATCH_SIZE}")
    print(f"   GPU Available: {torch.cuda.is_available()}")

    try:
        # Start training (this will resume automatically if interrupted)
        model, results = trainer.train_model(
            epochs=EPOCHS,
            img_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            resume=True  # Set to False if you want to start fresh
        )

        print("🎉 Training completed successfully!")

        # Display training plots if available
        plots_dir = f"{PROJECT_ROOT}/beverage_detection_training"
        if os.path.exists(plots_dir):
            print(f"📊 Training plots saved in: {plots_dir}")

            # Show results plot if available
            results_plot = f"{plots_dir}/results.png"
            if os.path.exists(results_plot):
                try:
                    display(Image(results_plot))
                except:
                    print(f"📈 Results plot available at: {results_plot}")

        # Quick validation check if training completed
        if 'model' in locals():
            print("\n🔍 Quick Model Validation:")
            val_results = model.val()
            print(f"   Validation mAP@0.5: {val_results.box.map50:.4f}")
            print(f"   Validation mAP@0.5:0.95: {val_results.box.map:.4f}")

    except KeyboardInterrupt:
        print("⏸️  Training interrupted by user")
        print("💾 Progress has been saved. Run this cell again to resume.")

    except Exception as e:
        print(f"❌ Training error: {e}")
        print("💾 Progress has been saved. Check the error and run again to resume.")

else:
    print("❌ Please update PROJECT_ROOT and yaml_path variables at the top of this code")
    print("📝 Set your actual paths and run again")

✅ Trainer initialized and ready!

🚀 Starting YOLO Training...
⚙️  Configuration:
   Epochs: 50
   Image Size: 640
   Batch Size: 16
   GPU Available: True
🚀 Starting fresh training
🖥️  Using device: cuda
🎯 Training Configuration:
   data: /content/drive/MyDrive/BeverageDetection/dataset/data.yaml
   epochs: 50
   imgsz: 640
   batch: 16
   device: cuda
   workers: 4
   project: /content/drive/MyDrive/BeverageDetection
   name: beverage_detection_training
   save_period: 10
   patience: 50
   save: True
   val: True
   plots: True
   verbose: True
   cache: True
   optimizer: SGD
   lr0: 0.01
   momentum: 0.937
   weight_decay: 0.0005
   warmup_epochs: 3.0
   warmup_momentum: 0.8
   warmup_bias_lr: 0.1

⏰ Training started at: 2025-05-27 17:36:46
Ultralytics 8.3.145 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosai

train: Scanning /content/drive/MyDrive/BeverageDetection/dataset/train/labels... 3828 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3828/3828 [01:15<00:00, 50.98it/s] 

train: /content/drive/MyDrive/BeverageDetection/dataset/train/images/1673_jpg.rf.4abcb7fbf3de5a342c7f00c19da35270.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/BeverageDetection/dataset/train/images/2661_jpg.rf.010b0d5e25cbf7c09a8b66ee0a767510.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/BeverageDetection/dataset/train/images/2877_jpg.rf.b19cb3e8e461f93d0401e96986e1f082.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/BeverageDetection/dataset/train/images/2969_jpg.rf.be9e3ad7d50e244cb6e6e294e5b72c09.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/BeverageDetection/dataset/train/images/3029_jpg.rf.1356660ffb1fb0b5df9184f52cd67362.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/BeverageDetection/dataset/train/images/3037_jpg.rf.ef7fc1a7621a813dddf1a21e0d8dba16.jpg: 1 duplicate labels removed
train: /content/drive/MyDrive/BeverageDetection/dataset/train/images/3090_jpg.rf.84f7e6b279b64b58ae9b696c55d6f7b8.jpg: 1 dup

train: New cache created: /content/drive/MyDrive/BeverageDetection/dataset/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 178, len(boxes) = 57314. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (3.0GB RAM): 100%|██████████| 3828/3828 [01:22<00:00, 46.37it/s] 


...

val: Scanning /content/drive/MyDrive/BeverageDetection/dataset/valid/labels.cache... 1094 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1094/1094 [00:00<?, ?it/s]

val: /content/drive/MyDrive/BeverageDetection/dataset/valid/images/2815_jpg.rf.263b9fd7ba997f6278fbc88329ae4bce.jpg: 1 duplicate labels removed
val: /content/drive/MyDrive/BeverageDetection/dataset/valid/images/2865_jpg.rf.3020ef0dda8fb2e35f6ac8c4edfd3164.jpg: 1 duplicate labels removed
val: /content/drive/MyDrive/BeverageDetection/dataset/valid/images/4021_jpg.rf.3516dd531ebfffd96b025e25fd67e363.jpg: 1 duplicate labels removed


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.9GB RAM): 100%|██████████| 1094/1094 [00:24<00:00, 44.52it/s] 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 69/69 [00:15<00:00,  4.41it/s]


                   all       1094      16535      0.715      0.736      0.721      0.584
     7-UP-200-ML-GLASS         53        122      0.745       0.74      0.797      0.619
      7-UP-2250-ML-PET        163        236      0.902      0.893      0.935      0.796
       7-UP-250-ML-PET        364        788      0.916      0.982      0.982      0.775
     7-UP-300-ML-GLASS          7         18      0.264     0.0823      0.179      0.119
       7-UP-500-ML-PET        207        493      0.925      0.951      0.975       0.81
       7-UP-600-ML-PET        117        192      0.519      0.651      0.583      0.486
       7-UP-750-ML-PET        136        237      0.614      0.865      0.698      0.619
       7-UP-CAN-250-ML         20         37      0.327      0.486      0.342      0.309
       7-UP-CAN-300-ML         44        109      0.725      0.941      0.786      0.648
      7-UP-PET-1250-ML        186        246       0.89      0.927      0.946      0.827
      AQUAFINA-1000-M

In [ ]:
!pip install --upgrade torch torchvision torchaudio numpy


In [ ]:
# Comprehensive Model Evaluation
import json
import pandas as pd
from ultralytics import YOLO

MODEL_PATH = "/content/drive/MyDrive/BeverageDetection/models/best_models/best_beverage_model.pt"
DATASET_YAML = "/content/drive/MyDrive/BeverageDetection/dataset/data.yaml"

if os.path.exists(MODEL_PATH):
    model = YOLO(MODEL_PATH)

    print("📊 COMPREHENSIVE MODEL EVALUATION")
    print("=" * 50)

    # 1. Overall Validation Metrics
    print("\n1️⃣ OVERALL PERFORMANCE")
    val_results = model.val(data=DATASET_YAML)

    print(f"📈 Key Metrics:")
    print(f"   • mAP@0.5 (IoU=0.5): {val_results.box.map50:.4f}")
    print(f"   • mAP@0.5:0.95 (IoU=0.5-0.95): {val_results.box.map:.4f}")
    print(f"   • Precision: {val_results.box.mp:.4f}")
    print(f"   • Recall: {val_results.box.mr:.4f}")
    print(f"   • F1-Score: {2 * (val_results.box.mp * val_results.box.mr) / (val_results.box.mp + val_results.box.mr):.4f}")

    # 2. Per-Class Performance
    print(f"\n2️⃣ PER-CLASS PERFORMANCE")
    if hasattr(val_results.box, 'ap_class_index') and len(val_results.box.ap_class_index) > 0:
        class_names = model.names
        for i, class_idx in enumerate(val_results.box.ap_class_index):
            class_name = class_names[class_idx]
            ap50 = val_results.box.ap50[i] if i < len(val_results.box.ap50) else 0
            ap = val_results.box.ap[i] if i < len(val_results.box.ap) else 0
            print(f"   📱 {class_name}:")
            print(f"      • AP@0.5: {ap50:.4f}")
            print(f"      • AP@0.5:0.95: {ap:.4f}")

    # 3. Model Information
    print(f"\n3️⃣ MODEL INFORMATION")
    print(f"   • Model Type: YOLOv8")
    print(f"   • Classes: {len(model.names)}")
    print(f"   • Class Names: {list(model.names.values())}")
    print(f"   • Parameters: {sum(p.numel() for p in model.model.parameters()):,}")

    # 4. Speed Benchmarks
    print(f"\n4️⃣ SPEED BENCHMARK")
    print("⏱️  Running speed test...")
    speed_results = model.val(data=DATASET_YAML, verbose=False)
    if hasattr(speed_results, 'speed'):
        print(f"   • Preprocess: {speed_results.speed['preprocess']:.2f}ms")
        print(f"   • Inference: {speed_results.speed['inference']:.2f}ms")
        print(f"   • Postprocess: {speed_results.speed['postprocess']:.2f}ms")
        total_time = sum(speed_results.speed.values())
        print(f"   • Total per image: {total_time:.2f}ms")
        print(f"   • FPS: {1000/total_time:.1f}")

    # 5. Training History (if available)
    print(f"\n5️⃣ TRAINING HISTORY")
    training_log_path = "/content/drive/MyDrive/BeverageDetection/training_log.json"
    if os.path.exists(training_log_path):
        with open(training_log_path, 'r') as f:
            training_data = json.load(f)
            print(f"   • Training completed: {training_data.get('timestamp', 'Unknown')}")
            print(f"   • Final epoch: {training_data.get('epoch', 'Unknown')}")
            if 'metrics' in training_data:
                metrics = training_data['metrics']
                print(f"   • Final mAP@0.5: {metrics.get('map50', 'N/A')}")
                print(f"   • Final mAP@0.5:0.95: {metrics.get('map', 'N/A')}")

    # 6. Performance Interpretation
    print(f"\n6️⃣ PERFORMANCE INTERPRETATION")
    map50 = val_results.box.map50
    if map50 >= 0.8:
        performance = "🟢 EXCELLENT"
    elif map50 >= 0.6:
        performance = "🟡 GOOD"
    elif map50 >= 0.4:
        performance = "🟠 FAIR"
    else:
        performance = "🔴 NEEDS IMPROVEMENT"

    print(f"   Overall Performance: {performance}")
    print(f"   mAP@0.5 of {map50:.3f} means:")
    print(f"   • The model correctly detects objects {map50*100:.1f}% of the time")
    print(f"   • At IoU threshold of 0.5 (50% overlap required)")

    # 7. Recommendations
    print(f"\n7️⃣ RECOMMENDATIONS")
    if map50 < 0.5:
        print("   🔧 Model needs improvement:")
        print("      • Increase training epochs")
        print("      • Add more training data")
        print("      • Check data quality and annotations")
        print("      • Try different augmentations")
    elif map50 < 0.7:
        print("   ⚡ Model is decent but can be improved:")
        print("      • Fine-tune hyperparameters")
        print("      • Add more diverse training data")
        print("      • Consider ensemble methods")
    else:
        print("   ✅ Model performance is good!")
        print("      • Ready for deployment")
        print("      • Consider speed optimizations if needed")
        print("      • Test on real-world scenarios")

else:
    print(f"❌ Model not found: {MODEL_PATH}")
    print("Please ensure training completed successfully")

In [ ]:

import os
import shutil
import torch
import json
from datetime import datetime
from ultralytics import YOLO
from IPython.display import display, Image

class BeverageDetectionTrainer:
    def __init__(self, dataset_yaml_path, project_root):
        self.dataset_yaml_path = dataset_yaml_path
        self.project_root = project_root
        self.checkpoints_path = os.path.join(project_root, "models/checkpoints")
        self.best_models_path = os.path.join(project_root, "models/best_models")
        self.training_log_path = os.path.join(project_root, "training_log.json")

        os.makedirs(self.checkpoints_path, exist_ok=True)
        os.makedirs(self.best_models_path, exist_ok=True)

    def save_training_state(self, epoch, model_path, metrics):
        state = {
            "epoch": epoch,
            "model_path": model_path,
            "metrics": metrics,
            "timestamp": datetime.now().isoformat(),
            "dataset_path": self.dataset_yaml_path
        }
        with open(self.training_log_path, "w") as f:
            json.dump(state, f, indent=2)
        print(f"💾 Saved training state at epoch {epoch}")

    def load_training_state(self):
        if os.path.exists(self.training_log_path):
            with open(self.training_log_path, "r") as f:
                state = json.load(f)
            print(f"📂 Loaded previous training state from epoch {state['epoch']}")
            return state
        return None

    def train_model(self, epochs=150, img_size=416, batch_size=8, resume=True):
        if not os.path.exists(self.dataset_yaml_path):
            raise FileNotFoundError(f"❌ Dataset YAML not found: {self.dataset_yaml_path}")

        previous_state = self.load_training_state() if resume else None

        if previous_state and os.path.exists(previous_state["model_path"]):
            print(f"🔄 Resuming training from {previous_state['model_path']}")
            model = YOLO(previous_state["model_path"])
            start_epoch = previous_state["epoch"]
        else:
            print("🚀 Starting fresh training with base model (yolov8n.pt)")
            model = YOLO("yolov8n.pt")
            start_epoch = 0

        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🖥️  Training on: {device}")

        train_params = {
            "data": self.dataset_yaml_path,
            "epochs": epochs,
            "imgsz": img_size,
            "batch": batch_size,
            "device": device,
            "workers": 4 if device == "cuda" else 2,
            "project": self.project_root,
            "name": "beverage_detection_training",
            "save_period": 10,
            "patience": 50,
            "save": True,
            "val": True,
            "plots": False,
            "verbose": False,
            "cache": "ram",
            "optimizer": "SGD",
            "lr0": 0.01,
            "momentum": 0.937,
            "weight_decay": 0.0005,
            "warmup_epochs": 3.0,
            "warmup_momentum": 0.8,
            "warmup_bias_lr": 0.1
        }

        try:
            print(f"\n⏰ Training started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            results = model.train(**train_params)

            best_model_src = results.save_dir / "weights" / "best.pt"
            best_model_dst = os.path.join(self.best_models_path, "best_beverage_model.pt")
            if os.path.exists(best_model_src):
                shutil.copy(str(best_model_src), best_model_dst)
                print(f"📁 Best model saved to: {best_model_dst}")

            # Extract metrics if available
            final_metrics = {}
            try:
                if hasattr(results, "box"):
                    final_metrics = {
                        "map50": float(results.box.map50),
                        "map": float(results.box.map),
                        "precision": float(results.box.mp),
                        "recall": float(results.box.mr)
                    }
            except Exception as metric_err:
                print(f"⚠️ Metric extraction failed: {metric_err}")
                final_metrics = {"error": str(metric_err)}

            self.save_training_state(epochs, best_model_dst, final_metrics)

            print("\n🎉 Training complete!")
            print("📊 Final Evaluation:")
            for k, v in final_metrics.items():
                print(f"   {k.upper()}: {v:.4f}" if isinstance(v, float) else f"   {k}: {v}")

            return model, results

        except KeyboardInterrupt:
            print("\n⏸️  Training interrupted by user.")
            self._save_interrupted_state(model, start_epoch)
            raise

        except Exception as e:
            print(f"\n❌ Unexpected error: {e}")
            self._save_interrupted_state(model, start_epoch)
            raise

    def _save_interrupted_state(self, model, start_epoch):
        try:
            epoch = getattr(model.trainer, "epoch", start_epoch)
            checkpoint_path = os.path.join(self.checkpoints_path, f"interrupted_epoch_{epoch}.pt")
            if hasattr(model.trainer, "last") and os.path.exists(model.trainer.last):
                shutil.copy(model.trainer.last, checkpoint_path)
                self.save_training_state(epoch, checkpoint_path, {})
                print(f"💾 Checkpoint saved at epoch {epoch}")
        except Exception as e:
            print(f"⚠️ Failed to save interrupted state: {e}")

    def validate_model(self, model_path=None):
        model_path = model_path or os.path.join(self.best_models_path, "best_beverage_model.pt")
        if not os.path.exists(model_path):
            print(f"❌ Model file not found: {model_path}")
            return None

        print(f"🔍 Validating model at: {model_path}")
        model = YOLO(model_path)
        results = model.val(data=self.dataset_yaml_path)

        print("\n📈 Validation Metrics:")
        print(f"   mAP@0.5:     {results.box.map50:.4f}")
        print(f"   mAP@0.5:0.95:{results.box.map:.4f}")
        print(f"   Precision:   {results.box.mp:.4f}")
        print(f"   Recall:      {results.box.mr:.4f}")

        return results
